In [ ]:
import torch
import torch.nn as nn
import torch.optim
import numpy as np
import traci
import optparse
import os
import sys
import pandas as pd
import json
import tqdm
from collections import deque

from sumolib import checkBinary  # noqa
import traci  # noqa

#Only works for micro simulation (1 traffic light)
class SumoEnvrionment:
    def __init__(self, gui = True, total_phases, buffer_size = 12):
        self.total_phases = total_phases
        self.buffer_size = buffer_size
        self.gui = gui

        self.lanes_dict = {}
        self.lane_IDs = traci.lanearea.getIDList()
        for lane in self.lane_IDs:
            self.lanes_dict[lane]=[]

        self.start_program()

    def start_program(self):
        options = self.get_options()
        options.nogui = self.gui

        # this script has been called from the command line. It will start sumo as a server, then connect and run
        if options.nogui:
            sumoBinary = checkBinary('sumo')
        else:
            sumoBinary = checkBinary('sumo-gui')

        # we need to import python modules from the $SUMO_HOME/tools directory
        if 'SUMO_HOME' in os.environ:
            tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
            sys.path.append(tools)
        else:
            sys.exit("please declare environment variable 'SUMO_HOME'")

        traci.start([sumoBinary, "-c", "Simulation_Environment\Main Route Simulation\osm.sumocfg",
                             "--tripinfo-output", "Data\\tripinfo.xml"])

    def json_loader(self, path):
        with open(path,'r') as f:
            all_data ="".join([i for i in f])
            file = json.loads(all_data)
        return file
    
    def get_options(self):
        optParser = optparse.OptionParser()
        optParser.add_option("--nogui", action="store_true",
                            default=False, help="run the commandline version of sumo")
        options, args = optParser.parse_args()
        return options

    def update_lane_data(self):
        for lane in self.lane_IDs:
            lane_data = traci.lanearea.getLastStepHaltingNumber(lane)
            self.lanes_dict[lane].append(lane_data)
        return self.lanes_dict
    
    def save_vehicle_ids(self, lanes_dict):
        vehicle_ids=pd.DataFrame(lanes_dict)
        vehicle_ids['Steps'] = np.arange(0,len(lanes_dict[lane_IDs[0]]),1)
        vehicle_ids.to_csv('test.csv')

    def get_reward(self):
        all_speed = []
        e2_detectors = traci.lanearea.getIDList()

        for detector in e2_detectors:
            speed = traci.lanearea.getLastStepMeanSpeed(detector)
            all_speed.append(speed)
        
        return speed

    def get_state(self):
        e2_detectors = traci.lanearea.getIDList()

        queues = [traci.lanearea.getLastStepVehicleNumber(detector) for detector in e2_detectors] #gets the queus in detectors
        traffic_light = traci.trafficlight.getIDList()[0] # since there is only one traffic light

        tl_phase = traci.trafficlight.getPhase(traffic_light)
        one_hot_vector_tl_phase = np.eye(self.total_phases)[tl_phase]

        return np.hstack([queues, one_hot_vector_tl_phase])
    
    def step(self):
        self.lanes_dict = self.update_lane_data()
        vehicle_ids=pd.DataFrame(self.lanes_dict)
        vehicle_ids['Steps'] = np.arange(0,len(self.lanes_dict[self.lane_IDs[0]]),1)
        vehicle_ids.to_csv('test.csv')


class Net(nn.Mdule):
    def __init__(self, total_phases):
        super().__init__()
        self.total_phases = total_phases
        self.fc1 = nn.Linear(8,1000)
        self.fc2 = nn.Linear(1000,500)
        self.fc3 = nn.Linear(500,self.total_phases)

    def forward(self, x):
        #Neural Network Forward Pass Layers
        x = nn.ReLU(self.fc1(x))
        x = nn.BatchNorm1d(x)
        x = nn.ReLU(self.fc2(x))
        x = nn.Relu(self.fc3(x))
        return x

def train(self, train=True, epochs = 80, mem_size = 1000, batch_size = 200, sync_freq = 500):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = Net().to(device)
    
if __name__ == '__main__':



In [6]:
class A:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def add(self):
        return self.a + self.b

class B(A):
    def __init__(self,c):
        super().__init__(a,b)
        self.a = a
        self.b = b
        self.c = c
    def product(self,a,b,c):
        return self.a*self.b*self.c

b = B(1,2,3)

TypeError: __init__() takes 2 positional arguments but 4 were given